In [ ]:
# compute likelihood
import numpy as np
from scipy.stats import pearsonr
import scipy.stats


def normal_ll(y, y_hat, transform=None, gradient=1):
    '''Log likelihood for the normal distribution with change of variable
    
    The normal distribution is the formal likelihood for the mean squared error (MSE).
    

    Parameters
    ----------
    y : array_like
        Observations.
    y_hat : array_like
        Predictions.
    transform : function
        Change of variable transformation.
    gradient : function
        Gradient of the transform function.
        
    Proof
    -----
    https://www.statlect.com/probability-distributions/normal-distribution
    '''
    if transform is not None:
        y = transform(y)
        y_hat = transform(y_hat)
        
    e = y - y_hat
    n = len(e)
    sigma = e.std()
    log_gradient = np.sum(np.log(np.abs(gradient)))
    ll = -n * np.log(sigma) - n/2*np.log(2*np.pi) - 1/(2*sigma**2) * (e**2).sum() + log_gradient
    return ll


def laplace_ll(y, y_hat, transform=None, gradient=1):
    '''Log likelihood for Laplace distribution with change of variable
    
    The laplace distribution is the formal likelihood for the mean absolute
    error (MAE).
    
    Parameters
    ----------
    y : array_like
        Observations.
    y_hat : array_like
        Predictions.
    transform : function
        Change of variable transformation.
    gradient : function
        Gradient of the transform function.
    '''
    if transform is not None:
        y = transform(y)
        y_hat = transform(y_hat)
        
    e = (y - y_hat).abs()
    n = len(e)
    b = e.mean()
    log_gradient = np.sum(np.log(np.abs(gradient)))
    ll = -n * np.log(2*b) - 1/b * e.sum() + log_gradient
    return ll.sum()
                                   

def msre_ll(y, y_hat):
    '''Log likelihood for mean squared square-root error
    
    Parameters
    ----------
    y : array_like
    y_hat : array_like
    '''
    return normal_ll(y, y_hat, transform=lambda x: np.sqrt(x), gradient=-1/(2*np.sqrt(y)))


def mare_ll(y, y_hat):
    '''Log likelihood for mean absolute square-root error
    
    Parameters
    ----------
    y : array_like
    y_hat : array_like
    '''
    return laplace_ll(y, y_hat, transform=lambda x: np.sqrt(x), gradient=-1/(2*np.sqrt(y)))


def lognormal_ll(y, y_hat):
    '''Lognormal log likelihood
    
    The lognormal distribution is the formal likelihood for the mean squared
    log error (MSLE).
    
    Parameters
    ----------
    y : array_like
    y_hat : array_like
    '''
    return normal_ll(y, y_hat, transform=lambda x: np.log(x), gradient=1/y)


def mspe_ll(y, y_hat):
    '''Log likelhood for mean squared percentage error
    
    Parameters
    ----------
    y : array_like
    y_hat : array_like
    
    '''
    return normal_ll(y, y_hat, transform=lambda x: x/y, gradient=-1/(y**2)) 


def nse_ll(y, y_hat, group='gage_id'):
    '''Log likelihood for normalized squared error (NSE)
    
    NSE is equivalent to the Nash–Sutcliffe model efficiency coefficient.
    
    Parameters
    ----------
    y : array_like
    y_hat : array_like
    '''
    sigma_o = y.groupby('gage_id').transform(lambda x: x.std())
    return normal_ll(y, y_hat, transform=lambda x: x/sigma_o, gradient=1/sigma_o)


def loglaplace_ll(y, y_hat):
    '''Log likelihood for log Laplace distribution
    
    Parameters
    ----------
    y : array_like
    y_hat : array_like
    '''
    return laplace_ll(y, y_hat, transform=lambda x: np.log(x), gradient=1/y)


def uniform_ll(y, y_hat):
    '''Log likelihood for uniform distribution.
    
    The uniform log likelihood minimizes the maximum error.
    
    Parameters
    ----------
    y : array_like
    y_hat : array_like
    '''
    e = np.abs(y - y_hat)
    n = len(e)
    #ll = -n * np.log(e.max()-e.min()) # standard formulation
    ll = -n * np.log(e.max() - 0)
    return ll


def bernoulli_ll(y, y_hat, groupby=None):
    '''TODO and use within zi_ll
    
    Parameters
    ----------
    y : array_like
    y_hat : array_like
    '''
    pass



def zi_ll(y, y_hat, ll=normal_ll, threshold=0.01, groupby=None):
    ''' Zero-inflated log likelihood.
    
     Parameters
    ----------
    y : array_like
    y_hat : array_like
    ll : function
        Zero-inflated log likelihood 
    threshold : float
        Value below which is treated as zero
    groupby : string
        Optional groupby term (testing)
    '''
    y_o = y <= threshold
    y_hat_o = y_hat <= threshold
    
    if groupby is None:
        n1 = (y_o & y_hat_o).sum() # correct zero-flow prediction
        n2 = (y_o ^ y_hat_o).sum() # incorrect zero-flow prediction 
    else:
        n1 = (y_o & y_hat_o).groupby(groupby).sum() # correct zero-flow prediction
        n2 = (y_o ^ y_hat_o).groupby(groupby).sum() # incorrect zero-flow prediction

    n3 = (~y_o & ~y_hat_o) # correct flow predictions
    
    # fraction of correctly predicted zero flows
    rho = np.where( (n1+n2) == 0, 0, n1 / (n1 + n2))
    n_rho = 1-rho
    
    # n1 * np.log(rho) + n2 * np.log(1-rho)
    ll_zero = n1[rho!=0] * np.log(rho[rho!=0]) + n2[n_rho!=0]* np.log(n_rho[n_rho!=0])
    
    return ll_zero.sum() + ll(y[n3], y_hat[n3])


def zilognormal_ll(y, y_hat):
    '''Log likelihood for zero-inflated lognormal.
    
    Parameters
    ----------
    y : array_like
    y_hat : array_like
    '''
       
    return zi_ll(y, y_hat, ll=lognormal_ll, threshold=0.01)


def ziloglaplace_ll(y, y_hat):
    '''Log likelihood for zero-inflated laplace.
    
    Parameters
    ----------
    y : array_like
    y_hat : array_like
    '''
    return zi_ll(y, y_hat, ll=loglaplace_ll, threshold=0.01)

In [ ]:
 def compute_weights(series, base=np.e):
    '''Compute posterior weights
    
    Parameters
    ----------
    series : array_like
        Log likelihoods
    base: float
        Base of the logarithm used to compute log likelihood
    '''
    s = base**series
    return s/s.sum()

In [ ]:
observations = 1
predictions = 1
    
    # step 1: create a table of objective functions
objectives = {
    'U' : {'name':'uniformly distributed error', 'f':uniform_ll},
    'MSE' : {'name':'mean squared error', 'f':normal_ll},
    'NSE' : {'name':'normalized squared error', 'f':nse_ll},
    'MAE' : {'name': 'mean absolute error', 'f':laplace_ll},
    'MSPE' : {'name': 'mean squared percent error', 'f':mspe_ll},
    'MSLE' : {'name':'mean squared log error*', 'f':lognormal_ll},
    'MALE' : {'name':'mean absolute log error*', 'f':loglaplace_ll},
    #'ZMSLE' : {'name':'zero-inflated MSLE', 'f':zilognormal_ll},
    #'ZMALE' : {'name':'zero-inflated MALE', 'f':ziloglaplace_ll},
    'MARE' : {'name':'mean absolute square root error', 'f':mare_ll},
}

objectives_df = pd.DataFrame.from_dict(objectives, orient='index')

# step 2: compute the information in each objective function
for index, row in objectives_df.iterrows():
    # nats is the negative log likelihood or the info in the error
    obj_df.loc[index, 'bits'] = - row.f(observations, predictions)/len(df)/np.log(2)

# step 3: compute weights
obj_df['weight'] = compute_weights(-obj_df.bits, base=2)

# step 4: format output table

table = obj_df[['name','bits','weight']].sort_values('weight').round(2)#.rename(columns=names)

table['rank'] = len(table) - np.argsort(table['weight'])